In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
import plotly.graph_objects as go
import plotly.express as px
import numpy as np

In [ ]:
plt.rcParams['font.family'] = 'Malgun Gothic'

In [ ]:
plt.rcParams['axes.unicode_minus'] = False

In [ ]:
file_paths = {
    'ab_assignment': '/content/drive/MyDrive/로그 분석 프로젝트/ab_assignment.csv',
    'campaigns': '/content/drive/MyDrive/로그 분석 프로젝트/campaigns.csv',
    'chat_events': '/content/drive/MyDrive/로그 분석 프로젝트/chat_events.csv',
    'event_logs': '/content/drive/MyDrive/로그 분석 프로젝트/event_logs.csv',
    'payment_transactions': '/content/drive/MyDrive/로그 분석 프로젝트/payment_transactions.csv',
    'plan_history': '/content/drive/MyDrive/로그 분석 프로젝트/plan_history.csv',
    'push_events': '/content/drive/MyDrive/로그 분석 프로젝트/push_events.csv',
    'referral_events': '/content/drive/MyDrive/로그 분석 프로젝트/referral_events.csv',
    'users': '/content/drive/MyDrive/로그 분석 프로젝트/users.csv'
}

dataframes = {}
datetime_cols = {
    'ab_assignment': ['assigned_at'],
    'campaigns': [],
    'chat_events': ['sent_at'],
    'event_logs': ['event_timestamp'],
    'payment_transactions': ['event_time'],
    'plan_history': ['change_time'],
    'push_events': ['sent_at'],
    'referral_events': ['referral_time'],
    'users': ['created_at', 'last_seen_at', 'plan_start_date', 'plan_end_date']
}

for name, path in file_paths.items():
    try:
        df = pd.read_csv(path)
        # datetime 컬럼 변환
        for col in datetime_cols[name]:
            if col in df.columns:
                df[col] = pd.to_datetime(df[col], errors='coerce')
        # event_logs의 event_properties 파싱
        if name == 'event_logs' and 'event_properties' in df.columns:
            df['event_properties_parsed'] = df['event_properties'].apply(
                lambda x: json.loads(x) if pd.notnull(x) else {}
            )
        dataframes[name] = df
    except FileNotFoundError:
        dataframes[name] = pd.DataFrame()

In [ ]:
event_logs_df = dataframes.get('event_logs')
payment_transactions_df = dataframes.get('payment_transactions')

In [ ]:
signup_users_df = event_logs_df[event_logs_df['event_name'] == 'signup_completed'].copy()
signup_users_df = signup_users_df.sort_values('event_timestamp').drop_duplicates('user_id')
num_step1 = signup_users_df['user_id'].nunique()
print(f"1. 회원가입 완료 (`signup_completed`): {num_step1}명")

1. 회원가입 완료 (`signup_completed`): 22545명


In [ ]:
first_lesson_users_df = event_logs_df[event_logs_df['event_name'] == 'first_lesson_played'].copy()
first_lesson_users_df = first_lesson_users_df.sort_values('event_timestamp').drop_duplicates('user_id')

In [ ]:
merged_step1_2_df = pd.merge(
        signup_users_df[['user_id', 'event_timestamp']].rename(columns={'event_timestamp': 'signup_time'}),
        first_lesson_users_df[['user_id', 'event_timestamp']].rename(columns={'event_timestamp': 'first_lesson_time'}),
        on='user_id',
        how='inner'
    )

In [ ]:
merged_step1_2_df = merged_step1_2_df[merged_step1_2_df['first_lesson_time'] >= merged_step1_2_df['signup_time']]
num_step2 = merged_step1_2_df['user_id'].nunique()
print(f"2. 첫 강의 재생 (`first_lesson_played`): {num_step2}명")


2. 첫 강의 재생 (`first_lesson_played`): 19001명


In [ ]:
payment_completed_users_df = event_logs_df[event_logs_df['event_name'] == 'payment_completed'].copy()
payment_completed_users_df = payment_completed_users_df.sort_values('event_timestamp').drop_duplicates('user_id')

In [ ]:
merged_step2_3_df = pd.merge(
        merged_step1_2_df[['user_id', 'first_lesson_time']],
        payment_completed_users_df[['user_id', 'event_timestamp']].rename(columns={'event_timestamp': 'payment_time'}),
        on='user_id',
        how='inner'
    )
merged_step2_3_df = merged_step2_3_df[merged_step2_3_df['payment_time'] >= merged_step2_3_df['first_lesson_time']]
num_step3 = merged_step2_3_df['user_id'].nunique()
print(f"3. 결제 완료 (`payment_completed`): {num_step3}명")

3. 결제 완료 (`payment_completed`): 3766명


In [ ]:
funnel_steps = {
        '회원가입 완료': num_step1,
        '첫 강의 재생': num_step2,
        '결제 완료': num_step3
    }

In [ ]:
funnel_data = pd.DataFrame(list(funnel_steps.items()), columns=['Step', 'Users'])

In [ ]:
funnel_data['Step_Conversion_Rate'] = (funnel_data['Users'] / funnel_data['Users'].shift(1) * 100).fillna(100)
funnel_data['Overall_Conversion_Rate'] = (funnel_data['Users'] / funnel_data['Users'].iloc[0] * 100).fillna(100)

In [ ]:
print(funnel_data.to_string(float_format="%.2f"))

      Step  Users  Step_Conversion_Rate  Overall_Conversion_Rate
0  회원가입 완료  22545                100.00                   100.00
1  첫 강의 재생  19001                 84.28                    84.28
2    결제 완료   3766                 19.82                    16.70


In [ ]:
bottleneck_index = funnel_data['Step_Conversion_Rate'][1:].idxmin()
bottleneck_step = funnel_data.loc[bottleneck_index, 'Step']
bottleneck_conversion = funnel_data.loc[bottleneck_index, 'Step_Conversion_Rate']
bottleneck_from_step = funnel_data.loc[bottleneck_index - 1, 'Step']

In [ ]:
print(f"주요 병목 구간: '{bottleneck_from_step}' → '{bottleneck_step}'")
print(f"이 구간의 전환율은 {bottleneck_conversion:.2f}% 입니다. 이탈이 가장 크게 발생합니다.")

주요 병목 구간: '첫 강의 재생' → '결제 완료'
이 구간의 전환율은 19.82% 입니다. 이탈이 가장 크게 발생합니다.


In [ ]:
fig = go.Figure(go.Funnel(
        y=funnel_data['Step'],
        x=funnel_data['Users'],
        textinfo="value+percent initial+percent previous",
        textfont={"family": "Malgun Gothic", "size": 12},
        marker={"color": ["#66c2a5", "#fc8d62", "#8da0cb"]},
        connector={"line": {"color": "gray", "dash": "dot"}},
    ))

fig.update_layout(
        title={
            'text': 'GATI 캠퍼스 사용자 퍼널 분석: 회원가입 → 첫 강의 재생 → 결제 완료',
            'font': {'family': "Malgun Gothic", 'size': 20}
        },
        yaxis_title={
            'text': '퍼널 단계',
            'font': {'family': "Malgun Gothic", 'size': 14}
        },
        xaxis_title={
            'text': '사용자 수',
            'font': {'family': "Malgun Gothic", 'size': 14}
        },
        font=dict(family="Malgun Gothic"),
        hovermode="x unified"
    )

fig.show()

In [ ]:
users_df = dataframes.get('users')
event_logs_df = dataframes.get('event_logs')
payment_transactions_df = dataframes.get('payment_transactions')
ab_assignment_df = dataframes.get('ab_assignment')

In [ ]:
signup_users_base = event_logs_df[event_logs_df['event_name'] == 'signup_completed'].copy()
signup_users_base = signup_users_base.sort_values('event_timestamp').drop_duplicates('user_id')
signup_users_base = signup_users_base[['user_id', 'event_timestamp']].rename(columns={'event_timestamp': 'signup_time'})

first_lesson_users_base = event_logs_df[event_logs_df['event_name'] == 'first_lesson_played'].copy()
first_lesson_users_base = first_lesson_users_base.sort_values('event_timestamp').drop_duplicates('user_id')
first_lesson_users_base = first_lesson_users_base[['user_id', 'event_timestamp']].rename(columns={'event_timestamp': 'first_lesson_time'})


payment_completed_users_base = payment_transactions_df[payment_transactions_df['status'] == 'success'].copy()
payment_completed_users_base = payment_completed_users_base.sort_values('event_time').drop_duplicates('user_id')
payment_completed_users_base = payment_completed_users_base[['user_id', 'event_time']].rename(columns={'event_time': 'payment_time'})

In [ ]:
activated_users = pd.merge(
        signup_users_base,
        first_lesson_users_base,
        on='user_id',
        how='inner'
    )
activated_users = activated_users[activated_users['first_lesson_time'] >= activated_users['signup_time']].copy()
activated_users['activated'] = True

cohort_base_df = pd.merge(
        activated_users,
        payment_completed_users_base,
        on='user_id',
        how='left'
    )

cohort_base_df['is_converted'] = (cohort_base_df['payment_time'].notnull()) & \
                                     (cohort_base_df['payment_time'] >= cohort_base_df['first_lesson_time'])

cohort_base_df = pd.merge(cohort_base_df, users_df, on='user_id', how='left')
cohort_base_df = pd.merge(cohort_base_df, ab_assignment_df[['user_id', 'experiment_name', 'variant']], on='user_id', how='left')

print(cohort_base_df.head())
print(f"총 활성화된 사용자 수 (첫 강의 재생 완료): {cohort_base_df['user_id'].nunique()}명")
print(f"활성화 후 결제 완료한 사용자 수: {cohort_base_df['is_converted'].sum()}명")

            user_id         signup_time   first_lesson_time  activated  \
0  usr_d0bd4612db19 2024-12-01 00:36:00 2025-04-03 15:35:18       True   
1  usr_81392af876f0 2024-12-01 02:27:00 2025-04-02 01:00:50       True   
2  usr_ecc11c7a383f 2024-12-01 02:35:00 2025-04-04 18:29:00       True   
3  usr_dc550b6fee36 2024-12-01 03:13:00 2025-04-03 12:43:58       True   
4  usr_6c7a24bca741 2024-12-01 03:45:00 2025-04-19 03:17:12       True   

  payment_time  is_converted          created_at        last_seen_at  \
0          NaT         False 2024-12-01 00:36:00 2025-07-09 20:31:00   
1          NaT         False 2024-12-01 02:27:00 2025-07-19 04:29:00   
2          NaT         False 2024-12-01 02:35:00 2025-07-20 13:18:00   
3          NaT         False 2024-12-01 03:13:00 2025-07-21 12:33:00   
4   2025-06-02          True 2024-12-01 03:45:00 2025-07-14 16:40:00   

                      email acquisition_source  ...     plan_start_date  \
0   eungyeong88@example.org       facebook_ads 

In [ ]:
valid_cohort_base_df = cohort_base_df[cohort_base_df['created_at'].notnull()].copy()

In [ ]:
valid_cohort_base_df['signup_weekday'] = valid_cohort_base_df['created_at'].dt.day_name()
valid_cohort_base_df['signup_hour'] = valid_cohort_base_df['created_at'].dt.hour

In [ ]:
weekday_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
valid_cohort_base_df = valid_cohort_base_df[valid_cohort_base_df['signup_weekday'].isin(weekday_order)].copy()
valid_cohort_base_df['signup_weekday'] = pd.Categorical(valid_cohort_base_df['signup_weekday'], categories=weekday_order, ordered=True)

In [ ]:
conversion_by_day_hour = valid_cohort_base_df.groupby(['signup_weekday', 'signup_hour']).agg(
        total_activated_users=('user_id', 'nunique'),
        converted_users=('is_converted', 'sum')
    ).reset_index()

/tmp/ipython-input-632923145.py:1: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [ ]:
conversion_by_day_hour['conversion_rate'] = (
        conversion_by_day_hour['converted_users'] / conversion_by_day_hour['total_activated_users'] * 100
    ).replace([np.inf, -np.inf], np.nan).fillna(0)

conversion_by_day_hour_pivot = conversion_by_day_hour.pivot_table(
        index='signup_weekday', columns='signup_hour', values='conversion_rate'
    )

/tmp/ipython-input-743013552.py:5: FutureWarning:

The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior



In [ ]:
if not conversion_by_day_hour_pivot.empty:
        fig = px.imshow(conversion_by_day_hour_pivot,
                         x=conversion_by_day_hour_pivot.columns,
                         y=conversion_by_day_hour_pivot.index,
                         color_continuous_scale='Viridis',
                         title='가입 요일 및 시간별 결제 전환율 (%)',
                         labels={'x': '가입 시간 (시)', 'y': '가입 요일', 'color': '전환율 (%)'},
                         text_auto=True
                        )
        fig.update_layout(
            font_family="Malgun Gothic",
            title_font_size=20,
            xaxis_title_font_size=14,
            yaxis_title_font_size=14,
            xaxis_nticks=24
        )
        fig.show()

In [ ]:
print("\n--- 2. 유입 채널별 결제 전환율 분석 ---")
conversion_by_source = cohort_base_df.groupby('acquisition_source').agg(
        total_activated_users=('user_id', 'nunique'),
        converted_users=('is_converted', 'sum')
    ).reset_index()
conversion_by_source['conversion_rate'] = (
        conversion_by_source['converted_users'] / conversion_by_source['total_activated_users'] * 100
    ).replace([np.inf, -np.inf], np.nan).fillna(0) # 무한대 값 및 NaN을 0으로 처리
conversion_by_source = conversion_by_source.sort_values(by='conversion_rate', ascending=False)

fig = px.bar(conversion_by_source,
                 x='acquisition_source',
                 y='conversion_rate',
                 title='유입 채널별 결제 전환율 (%)',
                 labels={'acquisition_source': '유입 채널', 'conversion_rate': '결제 전환율 (%)'},
                 text_auto=True,
                 color='conversion_rate',
                 color_continuous_scale=px.colors.sequential.Plasma # 색상 스케일
                )
fig.update_layout(
        font_family="Malgun Gothic",
        title_font_size=20,
        xaxis_title_font_size=14,
        yaxis_title_font_size=14,
        xaxis_tickangle=-45
    )
fig.show()


--- 2. 유입 채널별 결제 전환율 분석 ---


In [ ]:
print("\n--- 3. 디바이스별 결제 전환율 분석 ---")
conversion_by_device = cohort_base_df.groupby('device_type').agg(
        total_activated_users=('user_id', 'nunique'),
        converted_users=('is_converted', 'sum')
    ).reset_index()
conversion_by_device['conversion_rate'] = (
        conversion_by_device['converted_users'] / conversion_by_device['total_activated_users'] * 100
    ).replace([np.inf, -np.inf], np.nan).fillna(0)
conversion_by_device = conversion_by_device.sort_values(by='conversion_rate', ascending=False)

fig = px.bar(conversion_by_device,
                 x='device_type',
                 y='conversion_rate',
                 title='디바이스별 결제 전환율 (%)',
                 labels={'device_type': '디바이스 종류', 'conversion_rate': '결제 전환율 (%)'},
                 text_auto=True,
                 color='conversion_rate',
                 color_continuous_scale=px.colors.sequential.Mint # 색상 스케일
                )
fig.update_layout(
        font_family="Malgun Gothic",
        title_font_size=20,
        xaxis_title_font_size=14,
        yaxis_title_font_size=14
    )
fig.show()



--- 3. 디바이스별 결제 전환율 분석 ---


In [ ]:
print("\n--- 4. 성별 결제 전환율 분석 ---")
conversion_by_gender = cohort_base_df.groupby('gender').agg(
        total_activated_users=('user_id', 'nunique'),
        converted_users=('is_converted', 'sum')
    ).reset_index()
conversion_by_gender['conversion_rate'] = (
        conversion_by_gender['converted_users'] / conversion_by_gender['total_activated_users'] * 100
    ).replace([np.inf, -np.inf], np.nan).fillna(0)
conversion_by_gender = conversion_by_gender.sort_values(by='conversion_rate', ascending=False)

fig = px.bar(conversion_by_gender,
                 x='gender',
                 y='conversion_rate',
                 title='성별 결제 전환율 (%)',
                 labels={'gender': '성별', 'conversion_rate': '결제 전환율 (%)'},
                 text_auto=True,
                 color='conversion_rate',
                 color_continuous_scale=px.colors.sequential.Peach
                )
fig.update_layout(
        font_family="Malgun Gothic",
        title_font_size=20,
        xaxis_title_font_size=14,
        yaxis_title_font_size=14
    )
fig.show()


--- 4. 성별 결제 전환율 분석 ---


In [ ]:
print("\n--- 5. 연령대별 결제 전환율 분석 ---")
age_group_order = sorted(cohort_base_df['age_group'].unique(), key=lambda x: int(x.replace('s', '')) if isinstance(x, str) and x.endswith('s') else x)
cohort_base_df['age_group'] = pd.Categorical(cohort_base_df['age_group'], categories=age_group_order, ordered=True)

conversion_by_age_group = cohort_base_df.groupby('age_group').agg(
        total_activated_users=('user_id', 'nunique'),
        converted_users=('is_converted', 'sum')
    ).reset_index()
conversion_by_age_group['conversion_rate'] = (
        conversion_by_age_group['converted_users'] / conversion_by_age_group['total_activated_users'] * 100
    ).replace([np.inf, -np.inf], np.nan).fillna(0)
conversion_by_age_group = conversion_by_age_group.sort_values(by='age_group') # 연령대 순서로 다시 정렬

fig = px.bar(conversion_by_age_group,
                 x='age_group',
                 y='conversion_rate',
                 title='연령대별 결제 전환율 (%)',
                 labels={'age_group': '연령대', 'conversion_rate': '결제 전환율 (%)'},
                 text_auto=True,
                 color='conversion_rate',
                 color_continuous_scale=px.colors.sequential.PuBu # 색상 스케일
                )
fig.update_layout(
        font_family="Malgun Gothic",
        title_font_size=20,
        xaxis_title_font_size=14,
        yaxis_title_font_size=14
    )
fig.show()
print("연령대별 결제 전환율 바 차트가 생성되었습니다.")
print("\n" + "="*80 + "\n")

    # 6. A/B 테스트 변형별 결제 전환율 (바 차트)
print("\n--- 6. A/B 테스트 변형별 결제 전환율 분석 ---")
conversion_by_variant = cohort_base_df.groupby('variant').agg(
        total_activated_users=('user_id', 'nunique'),
        converted_users=('is_converted', 'sum')
    ).reset_index()
conversion_by_variant['conversion_rate'] = (
        conversion_by_variant['converted_users'] / conversion_by_variant['total_activated_users'] * 100
    ).replace([np.inf, -np.inf], np.nan).fillna(0)
conversion_by_variant = conversion_by_variant.sort_values(by='conversion_rate', ascending=False)

fig = px.bar(conversion_by_variant,
                 x='variant',
                 y='conversion_rate',
                 title='A/B 테스트 변형별 결제 전환율 (%)',
                 labels={'variant': 'A/B 테스트 변형', 'conversion_rate': '결제 전환율 (%)'},
                 text_auto=True,
                 color='conversion_rate',
                 color_continuous_scale=px.colors.sequential.Viridis # 색상 스케일
                )
fig.update_layout(
        font_family="Malgun Gothic",
        title_font_size=20,
        xaxis_title_font_size=14,
        yaxis_title_font_size=14
    )
fig.show()
print("A/B 테스트 변형별 결제 전환율 바 차트가 생성되었습니다.")
print("\n" + "="*80 + "\n")


--- 5. 연령대별 결제 전환율 분석 ---


/tmp/ipython-input-48117576.py:8: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



연령대별 결제 전환율 바 차트가 생성되었습니다.



--- 6. A/B 테스트 변형별 결제 전환율 분석 ---


A/B 테스트 변형별 결제 전환율 바 차트가 생성되었습니다.




In [ ]:
users_completed_first_lesson = cohort_base_df['user_id'].nunique()
users_completed_payment_after_lesson = cohort_base_df['is_converted'].sum()

In [ ]:
overall_payment_conversion_rate = (users_completed_payment_after_lesson / users_completed_first_lesson) * 100 if users_completed_first_lesson > 0 else 0

In [ ]:
print("\n--- 주요 결제 전환율 (첫 강의 재생 완료 후 결제 완료) ---")
print(f"첫 강의 재생 완료 사용자 수: {users_completed_first_lesson}명")
print(f"첫 강의 재생 후 결제 완료 사용자 수: {users_completed_payment_after_lesson}명")
print(f"결제 전환율 (payment_completed / first_lesson_played): {overall_payment_conversion_rate:.2f}%")
print("이 전환율은 '첫 강의를 수강한 사람 중 결제 완료한 사람의 비율'을 의미합니다.")
print("--------------------------------------------------")


--- 주요 결제 전환율 (첫 강의 재생 완료 후 결제 완료) ---
첫 강의 재생 완료 사용자 수: 19001명
첫 강의 재생 후 결제 완료 사용자 수: 3346명
결제 전환율 (payment_completed / first_lesson_played): 17.61%
이 전환율은 '첫 강의를 수강한 사람 중 결제 완료한 사람의 비율'을 의미합니다.
--------------------------------------------------
